In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from keras.layers import *
from keras.models import Model
from tensorflow import keras
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-d456bb71-756c-5f6b-b13b-6793b90de8b7"

2023-06-29 13:27:41.901338: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 13:27:42.541342: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bigdata_user5/Desktop/envs/nobo/lib/python3.7/site-packages/cv2/../../lib64:/home/bigdata_user5/Desktop/envs/nobo/lib/
2023-06-29 13:27:42.541410: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_dataset_path = os.listdir('3Dfingerknuckle_with_Aug/3D_Finger_Knuckle_Database/segmented_data_down')

train_subject_types = os.listdir('3Dfingerknuckle_with_Aug/3D_Finger_Knuckle_Database/segmented_data_down')

input_images=[]
input_labels=[]

train_type = ['middlefinger']
train_set=['set1','set2','set3','set4','set5','set6']

def scan_folder(train_dataset_path,label):
    images = []
    labels=[]
    complete_path=[x[0] for x in os.walk(train_dataset_path)]
    for file_name in complete_path:
        #print(file_name)
        for fn in  os.listdir(file_name):
            # print(fn)
            if fn.endswith(".bmp"):
            # if it's a txt file, print its name (or do whatever you want)
                # print(file_name)
                #im = Image.open(file_name+'/'+fn)
                im = cv2.imread(file_name+'/'+fn)
                # im.show()
                img_arr=cv2.resize(im,(224,224))
                # img_arr=im 
                images.append(img_arr)
                labels.append(label)
                #print(im.size)
                #print(im.filename)
             
        
        else:
            current_path = "".join((train_dataset_path, "/", file_name))
            if os.path.isdir(current_path):
                # if we're checking a sub-directory, recursively call this method
                scan_folder(current_path)
    #print(images)
    return images,labels


for item1 in train_subject_types:
    all_train_subjects = os.listdir('3Dfingerknuckle_with_Aug/3D_Finger_Knuckle_Database/segmented_data_down' + '/' +item1)
    #print(all_subjects)
    for item2 in train_type:
        for item3 in train_set:
            path=str('3Dfingerknuckle_with_Aug/3D_Finger_Knuckle_Database/segmented_data_down'  + '/' +item1 +  '/session1'+ '/' +item2 + '/'+item3+ '/3D')
            img,label=scan_folder(path,item1)
            input_images.extend(img)
            input_labels.extend(label)
            #print(input_labels)

input_train_images=np.array(input_images)  
input_train_labels=np.array(input_labels)
print(len(input_train_images))
print(len(input_train_labels))
#print(input_train_images)

55860
55860


In [4]:
test_dataset_path = os.listdir('3Dfingerknuckle_with_Aug/3D_Finger_Knuckle_Database/segmented_data_down')

test_subject_types = os.listdir('3Dfingerknuckle_with_Aug/3D_Finger_Knuckle_Database/segmented_data_down')

input_images=[]
input_labels=[]

test_type = ['middlefinger']
test_set=['set1','set2','set3','set4','set5','set6']


def scan_folder(test_dataset_path,label):
    images = []
    labels=[]
    #print(test_dataset_path)
    complete_path=[x[0] for x in os.walk(test_dataset_path)]
    #print(complete_path)
    for file_name in complete_path:
        #print(file_name)
        for fn in  os.listdir(file_name) :
            # print(fn)
            if fn.endswith(".bmp"):
            # if it's a txt file, print its name (or do whatever you want)
                # print(file_name)
                #im = Image.open(file_name+'/'+fn)
                im = cv2.imread(file_name+'/'+fn)
                # im.show()
                img_arr=cv2.resize(im,(224,224))
                #img_arr=im
                images.append(img_arr)
                labels.append(label)
             
        
        else:
            current_path = "".join((test_dataset_path, "/", file_name))
            if os.path.isdir(current_path):
                # if we're checking a sub-directory, recursively call this method
                scan_folder(current_path)
    #print(images)
    return images,labels


for item1 in train_subject_types:
    #all_train_subjects = os.listdir('data/3Dfingerknuckle/3D_Finger_Knuckle_Database/segmented_data_down' + '/' +item1)
    #print(all_train_subjects)
    for item2 in test_type:
        for item3 in test_set:
            path=str('3Dfingerknuckle_with_Aug/3D_Finger_Knuckle_Database/segmented_data_down'  + '/' +item1 +  '/session2'+ '/' +item2 + '/'+item3+ '/3D')
            img,label=scan_folder(path,item1)
            input_images.extend(img)
            input_labels.extend(label)
            # print(input_labels)

input_test_images=np.array(input_images) #,axis=0)
input_test_labels=np.array(input_labels)
# print(input_labels.shape)
# input_test_labels=np.concatenate(np.array(input_labels),axis=0)
print(input_test_images.shape)
print(input_test_labels.shape)
print(len(input_test_images))    
print(len(input_test_labels))

(7980, 224, 224, 3)
(7980,)
7980
7980


In [5]:
X_train = input_train_images
Y_train = input_train_labels

X_test = input_test_images
Y_test = input_test_labels

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(55860, 224, 224, 3)
(55860,)
(7980, 224, 224, 3)
(7980,)


In [6]:
# X_train = X_train/22.0
# X_test = X_test/35.0

# X_train[1,:]
# X_test[1,:]

from sklearn.preprocessing import LabelEncoder
lb= LabelEncoder()
Y_train = lb.fit_transform(Y_train)
Y_test = lb.fit_transform(Y_test)

In [7]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())

2.11.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


2023-06-29 13:28:07.715067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 13:28:08.446334: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:0 with 38541 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 3g.40gb, pci bus id: 0000:c2:00.0, compute capability: 8.0


In [8]:
# from keras_applications.EfficientNetV2B2 import , preprocess_input as resnet_preprocess_input
from keras.applications.xception import Xception, preprocess_input as xception_preprocess_input
from keras.layers import Concatenate, GlobalAveragePooling2D, Dense, Dropout, Input
from keras.models import Model
# from keras.utils import plot_model

# create ResNet50 and Xception models
efficientnet_model = tf.keras.applications.EfficientNetV2B2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
xception_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# create input tensor
input_tensor = Input(shape=(224, 224, 3))

# preprocess input for ResNet50 and XceptionNet
efficientnet_processed = tf.keras.applications.efficientnet.preprocess_input(input_tensor)
xception_processed = xception_preprocess_input(input_tensor)

# extract features from both models
efficientnet_features = efficientnet_model(efficientnet_processed)
xception_features = xception_model(xception_processed)

# concatenate features
concatenated = Concatenate()([efficientnet_features, xception_features])

# add global average pooling layer
pooled = GlobalAveragePooling2D()(concatenated)

# add fully connected layers for classification
fc1 = Dense(512, activation='relu')(pooled)
dropout = Dropout(0.5)(fc1)
fc2 = Dense(190, activation='softmax')(dropout)

# create model
model = Model(inputs=input_tensor, outputs=fc2)

# plot model architecture
# plot_model(model, show_shapes=True, to_file='model.png')
model.summary()

2023-05-12 11:03:25.134578: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38541 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 3g.40gb, pci bus id: 0000:c2:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 tf.math.subtract (TFOpLambda)  (None, 224, 224, 3)  0           ['tf.math.truediv[0][0]']        
                                                                                                  
 efficientnetv2-b2 (Functional)  (None, 7, 7, 1408)  8769374     ['input_3[0][0]']            

In [9]:
from keras.callbacks import ModelCheckpoint

checkpoint_filepath = 'checkpoint/checkpoint_pv5.h5'

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)


model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)


In [10]:
history = model.fit(X_train, Y_train, epochs=50, validation_data =(X_test, Y_test) , batch_size=64 , callbacks=[checkpoint_callback])

Epoch 1/50


2023-05-12 11:04:26.796361: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/efficientnetv2-b2/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-05-12 11:04:31.825025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-05-12 11:04:33.238678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-12 11:04:33.476921: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f95f8012dc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-12 11:04:33.476952: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB MIG 3g.40gb, Compute Capability 8.0
2023-05-12 11:04:33.481746: I tensorflow/compiler/mlir/te

873/873 [==============================] - 468s 462ms/step - loss: 1.1725 - accuracy: 0.7386 - val_loss: 1.6966 - val_accuracy: 0.6868
Epoch 2/50
873/873 [==============================] - 397s 455ms/step - loss: 0.3514 - accuracy: 0.9129 - val_loss: 1.4079 - val_accuracy: 0.7298
Epoch 3/50
873/873 [==============================] - 397s 455ms/step - loss: 0.2789 - accuracy: 0.9299 - val_loss: 1.4252 - val_accuracy: 0.7416
Epoch 4/50
873/873 [==============================] - 397s 454ms/step - loss: 0.2350 - accuracy: 0.9402 - val_loss: 1.1662 - val_accuracy: 0.7853
Epoch 5/50
873/873 [==============================] - 397s 454ms/step - loss: 0.2100 - accuracy: 0.9448 - val_loss: 0.8312 - val_accuracy: 0.8492
Epoch 6/50
873/873 [==============================] - 395s 453ms/step - loss: 0.1948 - accuracy: 0.9498 - val_loss: 1.0690 - val_accuracy: 0.8023
Epoch 7/50
873/873 [==============================] - 395s 453ms/step - loss: 0.1626 - accuracy: 0.9575 - val_loss: 1.4807 - val_accura

In [14]:
model.save('checkpoint/MFKR_Model_Aug_pre_50epochs.hdf5')
np.save('checkpoint/MFKR_Model_training_history_50epochs.npy', history.history)

In [11]:
model.evaluate(X_test, Y_test, batch_size=64)

125/125 [==============================] - 13s 105ms/step - loss: 1.4229 - accuracy: 0.8960


[1.422884464263916, 0.8959899544715881]

In [8]:
from keras.models import load_model

saved_model = load_model('checkpoint/checkpoint_pv5.h5')

saved_model.evaluate(X_test, Y_test, batch_size=128)

2023-06-29 13:28:15.521245: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38541 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 3g.40gb, pci bus id: 0000:c2:00.0, compute capability: 8.0
2023-06-29 13:28:23.733589: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100


 1/63 [..............................] - ETA: 8:10 - loss: 0.1210 - accuracy: 0.9844

2023-06-29 13:28:28.563166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


63/63 [==============================] - 22s 221ms/step - loss: 1.1836 - accuracy: 0.9090


[1.1836411952972412, 0.9090225696563721]

In [9]:
from keras.models import load_model

model = load_model('checkpoint/MFKR_Model_Aug_pre_50epochs.hdf5')

In [10]:
model.evaluate(X_test, Y_test, batch_size=64)

125/125 [==============================] - 16s 104ms/step - loss: 1.4229 - accuracy: 0.8960


[1.422884464263916, 0.8959899544715881]

In [12]:
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
import matplotlib.pyplot as plt

y_pred = saved_model.predict(X_test)

y_pred = np.argmax(y_pred, axis=1)

cm= confusion_matrix(Y_test,y_pred)
# np.set_printoptions(precision=2)
cr = classification_report(Y_test, y_pred, digits=4)
print(cr)
print(cm)

2023-05-17 12:27:47.352857: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1201213440 exceeds 10% of free system memory.


250/250 [==============================] - 18s 61ms/step
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        42
           1     1.0000    0.5952    0.7463        42
           2     1.0000    1.0000    1.0000        42
           3     1.0000    0.9048    0.9500        42
           4     0.9333    1.0000    0.9655        42
           5     0.9545    1.0000    0.9767        42
           6     0.9545    1.0000    0.9767        42
           7     0.9524    0.9524    0.9524        42
           8     1.0000    1.0000    1.0000        42
           9     1.0000    1.0000    1.0000        42
          10     0.9767    1.0000    0.9882        42
          11     0.7949    0.7381    0.7654        42
          12     1.0000    0.9524    0.9756        42
          13     0.8936    1.0000    0.9438        42
          14     0.9767    1.0000    0.9882        42
          15     1.0000    1.0000    1.0000        42
          16     0.8235 

/home/bigdata_user5/Desktop/envs/nobo/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bigdata_user5/Desktop/envs/nobo/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bigdata_user5/Desktop/envs/nobo/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [16]:
from sklearn.metrics import confusion_matrix

# assume y_true is the true labels and y_pred is the predicted labels
cm = confusion_matrix(Y_test, y_pred)

# calculate total false positives and true positives
fp = np.sum(cm, axis=0) - np.diag(cm)
tp = np.diag(cm)

# calculate total false discovery rate
fdr = np.sum(fp) / (np.sum(fp) + np.sum(tp))

print("False discovery rate (FDR) of the model:", fdr)

False discovery rate (FDR) of the model: 0.09097744360902256


In [12]:
import numpy as np
from sklearn.metrics import confusion_matrix

# assume y_true is the true labels and y_pred is the predicted labels
cm = confusion_matrix(Y_test, y_pred)

# calculate total true positives and false negatives
tp = np.diag(cm)
fn = np.sum(cm, axis=1) - tp

# calculate total sensitivity (TPR)
sensitivity = np.sum(tp) / (np.sum(tp) + np.sum(fn))

print("Sensitivity (TPR) of the model:", sensitivity)


Sensitivity (TPR) of the model: 0.9090225563909774


In [13]:
import numpy as np
from sklearn.metrics import confusion_matrix

# assume y_true is the true labels and y_pred is the predicted labels
cm = confusion_matrix(Y_test, y_pred)

# calculate false positives (FP), false negatives (FN), true positives (TP), and true negatives (TN)
FP = np.sum(cm, axis=0) - np.diag(cm)
FN = np.sum(cm, axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = np.sum(cm) - (FP + FN + TP)

# calculate the False Positive Rate (FPR) and False Negative Rate (FNR)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)

# calculate the Crossover Error Rate (CER)
CER = (FPR + FNR) / 2

print("Crossover Error Rate (CER):", np.mean(CER))

Crossover Error Rate (CER): 0.04572940287226


In [14]:
FPR = np.sum(FP) / np.sum(FP + TN)
FNR = np.sum(FN) / np.sum(FN + TP)
FDR = np.sum(FP) / np.sum(FP + TP)

print("false positive rate for the model:", FPR)
print("false negative rate for the model:", FNR)
print(FDR)

false positive rate for the model: 0.00048136213549747385
false negative rate for the model: 0.09097744360902256
0.09097744360902256
